### 데이터파일

- https://www.data.go.kr/data/15045016/fileData.do

### 필요한 컬럼만 뽑아서 csv파일로 저장

- 구/동을 별도의 컬럼으로 추가


### csv 파일에서 구/동/상호명 을 검색어로 셀레니움을 이용해 네이버, 카카오, 구글의 평점, 리뷰수, 페이지 url을 수집함

```- 중복값 처리(name/ 좌표값을 기준으로 동일한 값은 drop) 프랜차이즈 존재하기 떄문에 인자를 하나 더 넣어서 처리했음
 df[df.duplicated(['name','x','y'])] 116개 중복데이터 존재함
 unduplicated_df=df.drop_duplicates(['name','x','y'], keep='first')
 unduplicated_df.to_csv('csv_data/df_fixed.csv',encoding='euc-kr')```

### 구글 데이터 수집 코드

In [ ]:
import pandas as pd
from selenium import webdriver

import time
import os

from csv import writer
import numpy as np

df1 = pd.read_csv('df_fixed_dong.csv', sep=',' , encoding='CP949')

# 셀레니움 속도 향상을 위해 아래를 진행했으나 유의미하진 않았음
# options = webdriver.ChromeOptions()
# options.add_experimental_option("excludeSwitches", ["enable-logging"])
# browser = webdriver.Chrome(options=options)

In [ ]:
#구글 구동시 랜덤하게 정보가 있음에도 정보없음으로 수집되었기에 이부분을 확인하며 데이터를 수집하였다.
#그를 위해 인덱스를 지정했으며 마지막으로 정보없음을 출력한 인덱스부터 다시 돌리며 출력하였다.


df=df1[43638:].copy() 
df = df[['name', 'address','street', 'cate1', 'phone']]
df.columns = ['name', 'address', 'address2', 'cate1', 'phone'] 

# 리뷰: rateNum, 평점: rate

driver = webdriver.Chrome('chromedriver.exe')
df['google_keyword'] = '부산' + " " + df['address2'] + " " + df['name'] # 부산 ~동 가게이름
rate_list = []
google_map_search_url = 'https://www.google.com/maps'
driver.get(google_map_search_url)

for i, keyword in enumerate(df['google_keyword'].tolist()):
    print(keyword, end=" ")
    try:
        driver.get(google_map_search_url)
        driver.find_element_by_name('q').send_keys(keyword)
        driver.find_element_by_id('searchbox-searchbutton').click()
        time.sleep(3)
        
        rate = driver.find_element_by_css_selector("#pane > div > div.widget-pane-content.cYB2Ge-oHo7ed > div > div > div.x3AX1-LfntMc-header-title > div.x3AX1-LfntMc-header-title-ma6Yeb-haAclf > div.x3AX1-LfntMc-header-title-ij8cu > div.x3AX1-LfntMc-header-title-ij8cu-haAclf > div > div.gm2-body-2.h0ySl-wcwwM-RWgCYc > div.OAO0-ZEhYpd-vJ7A6b.OAO0-ZEhYpd-vJ7A6b-qnnXGd > span > span > span").text
        driver.implicitly_wait(1)
        rateNum = driver.find_element_by_css_selector("#pane > div > div.widget-pane-content.cYB2Ge-oHo7ed > div > div > div.x3AX1-LfntMc-header-title > div.x3AX1-LfntMc-header-title-ma6Yeb-haAclf > div.x3AX1-LfntMc-header-title-ij8cu > div.x3AX1-LfntMc-header-title-ij8cu-haAclf > div > div.gm2-body-2.h0ySl-wcwwM-RWgCYc > span:nth-child(3) > span > span:nth-child(1) > span.OAO0-ZEhYpd-vJ7A6b.OAO0-ZEhYpd-vJ7A6b-qnnXGd > span:nth-child(1) > button").text
        driver.implicitly_wait(1)
        driver.find_element_by_css_selector('#pane > div > div.widget-pane-content.cYB2Ge-oHo7ed > div > div > div:nth-child(4) > div:nth-child(5) > button').click()
        driver.implicitly_wait(1)
        rate_href = driver.find_element_by_css_selector('#hoNUob-Sx9Kwc-m5SR9c > div.hoNUob-haAclf > div > div.modal-dialog-content > div > div > div.siAUzd-neVct > div.MmSeke-jCCvxc-hSRGPd > div:nth-child(2) > div.MmSeke-jCCvxc-hSRGPd-YPqjbf-haAclf > input').get_attribute("value")
        driver.implicitly_wait(1)
        print( rateNum + ", 평점:" + rate+ ' ' + ', 링크:'+ rate_href )
        
        rate_list.append([keyword, rate, rateNum, rate_href])
        driver.implicitly_wait(1)
    
    except:
        print('정보없음')
        rate_list.append([keyword, np.nan, np.nan, np.nan])
   
    time.sleep(1)

    
#원래는 rate_list에 데이터가 남아있어야하나 셀에 뜬 *가 계속 유지되며 데이터를 수집하여 restart 서버를 누를수 밖에 없었고
#아래를 추가해서 끊어서 csv파일로 저장시키며 작업했다.

    if ((i+1)%10)==0:
        df_rate= pd.DataFrame(rate_list, columns=['가게 이름', '평점', '리뷰', '링크'])
        df_rate.to_csv('rate_save_google_26500.csv', index = False)
        
#끊어서 저장하다보니 인력으로 하는 부분이 있었다.. 마지막 인덱스를 str.contain으로 찾으며 작업했다.
    
df_google = pd.DataFrame(rate_list, columns=['가게 이름', '평점', '리뷰', '링크']) 
df_google.to_csv('rate_save_google_end.csv', index = False)

## 네이버 데이터 수집 코드

In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

import time
import os
from csv import writer
import numpy as np

In [ ]:
df = pd.read_csv('df_fixed_dong.csv', sep=',' , encoding='CP949')

df = df[['name', 'address','street', 'cate1', 'phone']] 
df.columns = ['name', 'address', 'address2', 'cate1', 'phone'] 

# 리뷰: rateNum, 평점: rate

driver = webdriver.Chrome('chromedriver.exe')
df['naver_keyword'] = '부산' + " " + df['address2'] + " " + df['name'] # 부산 ~동 가게이름
rate_list = []
naver_map_search_url = 'https://map.naver.com/'
driver.get(naver_map_search_url)

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    try:
        naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}"
        driver.get(naver_map_search_url)
        time.sleep(3)
        
#네이버는 iframe 형식으로 되어있기 때문
# https://hogni.tistory.com/105
        driver.switch_to.frame('entryIframe')
        rate = driver.find_element_by_css_selector('#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR._2z4r0 > div > span._1Y6hi._1A8_M > em').text
        driver.implicitly_wait(1)
    #         rate = float(rate)
        count_review = driver.find_element_by_css_selector('#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR._2z4r0 > div > span:nth-child(2) > a > em').text
        driver.implicitly_wait(1)
    #         count_review = int(count_review)
        review_herf = driver.find_element_by_css_selector('#_btp\.share').get_attribute('data-url')
        print(f'사업장명 : {keyword}, 평점: {rate} , 리뷰: {count_review}, 링크: {review_herf}')
        rate_list.append([keyword, rate, count_review, review_herf])
        driver.implicitly_wait(1)
        
    except:
        print(f'사업장명 : {keyword} , 정보없음')
        rate_list.append([keyword, np.nan, np.nan, np.nan])
    time.sleep(1)     
   
#     driver.find_element_by_name('q').clear()    
    
    df_naver = pd.DataFrame(rate_list, columns=['사업장명', '평점', '리뷰', '링크']) 

# 

## 카카오 데이터 수집 코드

In [4]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
import numpy as np

In [ ]:
df = pd.read_csv('csv_dat/df_fixed_dong.csv', sep=',',encoding='euc-kr')
# df = df[['사업장명', '소재지전체주소','gu','dong','업태구분명', '소재지전화','좌표정보(x)','좌표정보(y)']] # 나중에 동으로 구분한 컬럼도 넣어야합니다.
# df.columns = ['name', 'address','district','street' ,'cate1', 'phone','x','y']
# df[df.duplicated(['name','x','y'])] 116개 중복데이터 존재함
# unduplicated_df=df.drop_duplicates(['name','x','y'], keep='first')
# unduplicated_df.to_csv('csv_data/df_fixed.csv',encoding='euc-kr')

In [ ]:
df = pd.read_csv('df_fixed_dong.csv', sep=',',encoding='euc-kr')

#셀레니움 속도 향상을 위해 암묵적대기로 진행함
driver = webdriver.Chrome('chromedriver.exe') # seconds
driver.implicitly_wait(1)
df['kakao_keyword'] = df['district']+''+df['street'] + " " + df['name']

search_list=[]

for i, keyword in enumerate(df['kakao_keyword'].tolist()):
    try:
        kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
        driver.get(kakao_map_search_url)
        name=keyword.split()[-1]
        rate = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea > span.score > em").text
        rateNum = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea > a > em").text
        url= driver.find_element_by_css_selector("#info\.search\.place\.list >li:nth-child(1)>div.rating.clickArea>a")
        url=url.get_attribute('href')

        search_list.append([name, rate, rateNum, url])

    except Exception as e1:
        print(str(i)+" 정보 없음")
        search_list.append([name,np.nan,np.nan,np.nan])
        pass
    
df_list = pd.DataFrame(search_list,columns=['name', '카카오 평점', '카카오 리뷰','카카오 URL'])
df_list[['name','카카오 평점','카카오 리뷰','카카오 URL']]


### 세개를 기존 파일에 합치기

- Projection 정의
- 중부원점(Bessel): 서울 등 중부지역 EPSG:2097

- WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326


In [ ]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
import folium
# Projection 정의
# 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
proj_1 = Proj(init='epsg:2097')

# WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
proj_2 = Proj(init='epsg:4326')

In [ ]:
file=input()
df=pd.read_csv(file+'.csv',encoding='UTF-8')

In [7]:
#좌표계를 folium 적용시키기 위해 변경해줌 EPSG:4326로

DataFrame = df.copy() #copy 를 이용하면 기존 데이터와 겹치지 않고 하나의 파일을 만들수 있는것. 

x_list = []
y_list = []

for idx, row in DataFrame.iterrows():
    x, y = row['x'], row['y']
    x_, y_ = transform(proj_1, proj_2, x, y)
    x_list.append(x_)
    y_list.append(y_)
    print(row['Unnamed: 0'])
    
df['lon'] = x_list
df['lat'] = y_list

NameError: name 'transform' is not defined

## 데이터 합치기

```
- 데이터프레임 내 중복 항목을 True False 로 반환
 df.duplicated(['name','district','street'])
- 합쳐진 데이터를 확인해야함으로 다시 한번 진행했다,
```

### df.info() 를 통해 개수와 컬럼 등을 확인하고 진행
- 합칠때 누락발생할 수 있으므로 개수 확인 index로 찾으므로 index 제대로 됐나 확인


- df[df['columns'].str.contains('가게이름')]
- 어디서까지 합칠건지 검색
- 마지막으로 출력된 결과 확인 후 +1

### 전체데이터, 앞에 합칠거, 뒤에 합칠거

```df_first=pd.read_csv('google_start-26557.csv')
df_first[df_first['가게 이름'].str.contains('모름지기연탄구이')]
df_first```

```df_dup_check_part=df_second[df_second.duplicated(['가게 이름'])]```

In [ ]:
#앞서 나온 검색 결과에다가 추가 이어붙이기/출력된곳확인 후 +1
result1 = pd.concat([result1,df_second[4000:]])
#결과를 csv
#전체 데이터에서 인덱스 확인후 제목 기입
result1.to_csv('google_late_final.csv')
result1
#중복데이터 처리하기
#합칠때 잘못해서 중복데이터 41개 발생함
#합칠때 발생한 문제이기에 합칠 파일별로 끊어서 개수를 확인하고 문제가 되는 부분을 찾아냈음


In [ ]:
# google_late_final.rename(columns={'가게 이름':'name'},inplace=True)

google_late_final['가게 이름']=google_late_final['가게 이름'].apply(lambda x : x.split(' ')[-1])
#가게 이름에서 열별 값 비교를 위해 구 동을 뺀 name만 추출

In [ ]:
#중복데이터 처리 후 원본 값과 비교를 위해 진행
df1['name']=df1['name'].apply(lambda x : x.split(' ')[-1])
df1
#전체에서  name 컬럼에 똑같이 적용해보기
df1['name']=df1['name'].apply(lambda x : x.split(' ')[-1])
df1
#원본과 합본이 일치함을 확인함
df1[df1['name'].eq(google_late_final['가게 이름'])==True]

### 세개 데이터를 합치기
- concat 함수 이용해서 옆으로 쭉 붙이기
```df_col=pd.concat([df_all,df_kakao,df_naver,df_google],axis=1)```

In [ ]:
#컬럼명 바꾸기 rename

df_naver.rename(columns={'평점':'네이버 평점','리뷰':'네이버 리뷰', '링크': '네이버 URL'},inplace=True)
df_naver

``` 
- 이름값을 기준으로 합치려고 했으나 알수 없는 문제가 발생해서 merge가 아닌 concat으로 합치기로 함
merge_left = pd.merge(df_fixed,df_list2, how='left', left_on='name', right_on='name')
merge_left
- 문제해결을 위해 원본과 합본에서 공통항목을 받아보았으나 답을 찾지 못함
N_list=set(df_fixed['name'].unique()) - set(df_list2['name'].unique())
- 접근법이 틀린거같음 ```

In [ ]:
#concat 함수 이용해서 옆으로 쭉 붙이기
df_col=pd.concat([df_all,df_kakao,df_naver,df_google],axis=1)

#필요한 열만 추출
df[['컬럼명1','컬럼명2','컬럼명3']]

#중복제거를 위한 columns 확인
df_col.columns.values

#불필요한 컬럼제거
DataFrame.drop([DataFrame.columns[0],DataFrame.columns[1],DataFrame.columns[8],DataFrame.columns[9]], axis=1)

In [ ]:
#위도 경도 결측치 제거 2개 결측값이 발견
DataFrame.info()
# WGS84 경위도 결측치가 없는건 잘못된 결과이며 중부원점 기준으로 결측치 확인 후 제거하겠음
# 혹시 모르니 제거전 값은 보존 
DataFrame=pd.read_csv('df_좌표결측치제거전4(43641존재).csv')

#'x' 컬럼을 기준으로 결측값을 행으로 제거함
DataFrame=DataFrame.dropna(subset=['x'])

In [ ]:
#nan 값 제거 후에는 인덱스가 맞지않는다 df=df.reset_index(drop=True)
DataFrame=DataFrame.reset_index(drop=True)

In [ ]:
#info()  확인해보니 결측값있음
#몇개안되서 출력해서 확인하고 채움
DataFrame[DataFrame['cate1'].isna()==True]

### cate1 종목 분류

#### 원하는 기능 구현이 어려워서 7개로 csv파일을 나누고 링크를 통해 들어가게 한다.

- 한식(한식, 김밥(도시락),식육(숯불구이),탕류(보신용),복어취급) 
- 중식(중국식) 
- 일식(일식,회집,횟집) 
- 양식(경양식,패밀리레스트랑) 
- 패스트푸드/치킨(호프/통닭,분식,통닭(치킨),패스트푸드 ) 
- 기타(기타,정종/대포집/소주방,외국음식전문점(인도,태국등),뷔페식,까페,냉면집,라이브카페,이동조리,전통찻집,키즈카페,출장조리,감성주점,일반조리판매,관광호텔,식품등 수입판매업)


In [8]:
# cate1 항목분류를 위해 값의 종류를 확인해봄
DataFrame['cate1'].value_counts() 

한식                 18505
기타                  4397
호프/통닭               3871
경양식                 3179
분식                  2895
식육(숯불구이)            2656
중국식                 1757
통닭(치킨)              1131
일식                  1046
정종/대포집/소주방          1016
회집                   926
횟집                   697
김밥(도시락)              358
탕류(보신용)              257
패스트푸드                188
외국음식전문점(인도,태국등)      177
뷔페식                  145
까페                   106
냉면집                  104
복어취급                  74
라이브카페                 39
패밀리레스트랑               27
이동조리                  23
키즈카페                  19
전통찻집                  19
출장조리                  13
감성주점                   7
관광호텔                   2
일반조리판매                 2
식품등 수입판매업              1
Name: cate1, dtype: int64

In [ ]:
df=pd.read_csv('df_102410.csv')


list_hansik=['한식', '김밥(도시락)', '식육(숯불구이)','탕류(보신용)','복어취급']
list_china=['중식(중국식)']
list_japan=['일식', '회집', '횟집']
list_family=['경양식', '패밀리레스트랑']
list_fast_chic=['호프/통닭','분식','통닭(치킨)','패스트푸드']
list_else=['기타','정종/대포집/소주방','외국음식전문점(인도,태국등)','뷔페식','까페','냉면집','라이브카페','이동조리','전통찻집','키즈카페','출장조리','감성주점','일반조리판매','관광호텔','식품등 수입판매업']


#위처럼 특정한 리스트 형태의 값들을 포함하는 행을 추출하고 싶다면 아래와 같이 isin 함수를 사용하는 것이 더욱 간편하다.
isin_hansik = df['cate1'].isin(['한식', '김밥(도시락)', '식육(숯불구이)','탕류(보신용)','복어취급'])
df_hansik=df.loc[isin_hansik]
df_hansik.drop(df_hansik.columns[0],axis=1,inplace=True)

# #값 제거 후에는 인덱스가 맞지않는다 df=df.reset_index(drop=True)
df_hansik=df_hansik.reset_index(drop=True)
df_hansik
df_hansik.to_csv('한식.csv')  

### folium 으로 시각화

In [9]:
import pandas as pd
import folium
#지도 바로 열어보려고 필요
import webbrowser
#마커클러스터와 검색창 플러그인을 이용할것임
from folium.plugins import MarkerCluster ,Search
import numpy as np


In [ ]:
name=str(input())
data_frame=pd.read_csv(name+'.csv',encoding='UTF-8')
data_frame1=data_frame.copy()

In [ ]:
#별점을 위해 데이터값에 df.fillna()
#np.nan으로 기입된 데이터도 있고 아닌데이터도 있음
#결측값이 별점 계산에 영향을 미치므로 0으로 일괄적으로 값을 줬다
data_frame1 = data_frame1.replace(np.nan,0)
data_frame1.info()

- 원하는 기능 구현을 위해 js 오픈소스를 적용하는데 너무 어려워서,, 한참헤맸다.
- 가장 쉬운 방법을 찾았다.


In [ ]:
map = folium.Map(location = [35.1856505,129.1], zoom_start=11)
marker_cluster=MarkerCluster().add_to(map)

for idx,row in data_frame1.iterrows():
    
#평균 평점 구하기
    a_rate=[row["카카오 평점"],row["구글 평점"],row["네이버 평점"]]
    vol=3

#     for i in range(3):
#         if a_rate.count(0)==i:
#             avg_rate=sum(a_rate)/(3-i)
#         elif a_rate.count(0)==3:
#             avg_rate=sum(a_rate)

    for i in range(3):
        if a_rate[i]<0.1:
            vol -= 1
    if vol!=0:
        avg_rate=sum(a_rate)/vol
    else:
        avg_rate=0
        
    if avg_rate >=3.5:
        rate_color ="red"
    elif avg_rate > 0:
        rate_color="orange"
    elif avg_rate ==0:
        #grey 가 아닌 gray였다..
        rate_color="gray"

#하이퍼링크 link0 카카오 link1 구글 link2 네이버
    a_href=[row["카카오 URL"],row["구글 URL"],row["네이버 URL"]]
    for i in range(3):
        if a_href[i] != 0:
            globals()['link{}'.format(i)]=f'<a href = {str(a_href[i])} target =_blink>링크</a>'
        else:
            globals()['link{}'.format(i)]='0'
            
    html = ("<h4> <b>" +str(row["name"]) +"</h4></b>"+str(row["cate1"])
            +"<br>카카오 : " +str(row["카카오 평점"])+'&nbsp'+str(row["카카오 리뷰"])+'&nbsp'+link0
            +"<br>구글 : "+str(row["구글 평점"])+'&nbsp'+str(row["구글 리뷰"])+'&nbsp'+link1
            +"<br>네이버 : "+str(row["네이버 평점"])+'&nbsp'+str(row["네이버 리뷰"])+'&nbsp'+link2)
                
    iframe = folium.IFrame(html)
    print_popup=folium.Popup(iframe,min_width=230 ,max_width=400)
    folium.Marker(location=[row['lat'],row['lon']],popup=print_popup,
                  icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(marker_cluster)

############################plugins-search / geojson을 이용하기 위해 csv 파일 json으로 변경 해주었다.
geo_json = {
  "type": "FeatureCollection",
  "features": [],
}

#geojson에서는 위도 경도가 다르게 입력된다.

for idx,row in data_frame1.iterrows():
    temp_dict = {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates":[row["lon"],row["lat"]],
        
      },"properties": {"name": row["name"]}
    }
    geo_json["features"].append(temp_dict)

#팝업이 겹쳐서 show=False를 줘서 레이어가 보이지 않게 했다
geojson_obj = folium.GeoJson(geo_json, show=False).add_to(map)


###서치박스 힘들었음
servicesearch = Search(
    layer=geojson_obj,
    search_label="name",
    placeholder='Search for a service',
    collapsed=False,
    search_zoom=20,

).add_to(map)

folium.LayerControl().add_to(map)

map
map.save(name+'.html')
webbrowser.open(name+'.html')

#### 다른 방법 markercluster +FeatureGroup
- 팝업아이콘이 가려버리는 문제가 발생했다.


In [ ]:


map = folium.Map(location = [35.1856505,129.1], zoom_start=12)
marker_cluster=MarkerCluster().add_to(map)

fg = folium.FeatureGroup(name='전체')
map.add_child(fg)

g1 = plugins.FeatureGroupSubGroup(fg, '한식')
map.add_child(g1)

g2 = plugins.FeatureGroupSubGroup(fg, '중식')
map.add_child(g2)

g3 = plugins.FeatureGroupSubGroup(fg, '일식')
map.add_child(g3)

g4 = plugins.FeatureGroupSubGroup(fg, '양식')
map.add_child(g4)

g5 = plugins.FeatureGroupSubGroup(fg, '패스트푸드/치킨')
map.add_child(g5)

g6 = plugins.FeatureGroupSubGroup(fg, '기타')
map.add_child(g6)

#cate의 list화
list_hansik=['한식', '김밥(도시락)', '식육(숯불구이)','탕류(보신용)','복어취급']
list_china=['중식(중국식)']
list_japan=['일식', '회집', '횟집']
list_family=['경양식', '패밀리레스트랑']
list_fast_chic=['호프/통닭','분식','통닭(치킨)','패스트푸드']
list_else=['기타','정종/대포집/소주방','외국음식전문점(인도,태국등)','뷔페식','까페','냉면집','라이브카페','이동조리','전통찻집','키즈카페','출장조리','감성주점','일반조리판매','관광호텔','식품등 수입판매업']



# temp_dict = {"type": "Feature",
#       "geometry": {
#         "type": "Point",
#         "coordinates":[row['lat'],row['lon']],
        
#       },"properties": {"name":row["name"]}}
    
for idx,row in data_frame1.iterrows():
    #마커 for 문안에서 돌려서 찍기
    
    
        
    if float(row["카카오 평점"])>=4:
        rate_color ="red"
    elif float(row["카카오 평점"])>=3:
        rate_color="orange"
    else:
        rate_color="yellow"

    x= f'<a href = {str(row["카카오 URL"])} target =_blink>링크</a>'
    x2= f'<a href = {str(row["구글 URL"])} target =_blink>링크</a>'
    x3= f'<a href = {str(row["네이버 URL"])} target =_blink>링크</a>'
    html = "<h4> <b>name :&nbsp" +str(row["name"]) +"</h4></b>"+"<br>"+str(row["cate1"])+"<br>카카오 : " +str(row["카카오 평점"])+'&nbsp'+str(row["카카오 리뷰"])+'&nbsp'+x+"<br>구글 : "+str(row["구글 평점"])+'&nbsp'+str(row["구글 리뷰"])+'&nbsp'+x2+"<br>네이버 : "+str(row["네이버 평점"])+'&nbsp'+str(row["카카오 리뷰"])+'&nbsp'+x3
    iframe = folium.IFrame(html)
    print_popup=folium.Popup(iframe,min_width=400,max_width=400)
    
    if row['cate1'] in list_hansik:
        folium.Marker([row['lat'],row['lon']],popup=print_popup,name=row['name'],icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(g1)    
    elif row['cate1'] in list_china:
        folium.Marker([row['lat'],row['lon']],popup=print_popup,icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(g2)
    elif row['cate1'] in list_japan:
        folium.Marker([row['lat'],row['lon']],popup=print_popup,icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(g3)        
    elif row['cate1'] in list_family:
        folium.Marker([row['lat'],row['lon']],popup=print_popup,icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(g4)
    elif row['cate1'] in list_fast_chic:
        folium.Marker([row['lat'],row['lon']],popup=print_popup,icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(g5)
    else:
        folium.Marker([row['lat'],row['lon']],popup=print_popup,icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(g6) 
    
#     geo_json = {
#   "type": "FeatureCollection",
#   "features": [],
#     }

    

#     geo_json["features"].append(temp_dict)
    
#     geojson_obj = folium.GeoJson(geo_json).add_to(map)
    
#     servicesearch = Search(
#         layer= geojson_obj,
#         search_label="name",
#         placeholder='Search for a service',
#         collapsed=False,
#     ).add_to(map)    
    
    
    #로드뷰그림 삽입가능할지?
   
#     print_popup =str(row["name"])+"<br>"+str(row["cate1"])+"<br>카카오" +str(row["카카오 평점"])+x+"<br>구글"+"<br>네이버"  
    
#     folium.Marker(location=[row['lat'],row['lon']],popup=print_popup,icon=folium.Icon(color=rate_color,icon='info-sign')).add_to(marker_cluster)
    
folium.LayerControl(collapsed=False).add_to(map) 


# searchbox
# geo_json = {
#   "type": "FeatureCollection",
#   "features": [],
# }
# for d in data_frame1.iterrows():
    





map

map.save('test.html')
webbrowser.open('test.html')

# m

### map.html에 링크버튼 만들고 별점 아이콘
- 

In [ ]:
#한참 헤맸다 진짜
#css html <a> <button> onclick location href  다 해봤는데 작동이 되지 않았다
#css로 만드니 버튼 클릭은 되는데 지도에서 버튼이 숨어버렸고 줌아웃할때만 반짝 나오는데
#z-index를 수정해도 고쳐지지 않았다.
#zoom in을 따와서 거기에 넣어서 클릭되게 했다. 

In [ ]:
body에 아래 항목을 추가했다.

In [ ]:
<div class="leaflet-control-container">
                    <div class="leaflet-bottom leaflet-right" style="margin-bottom: 71px">
                        <div class="leaflet-control-zoom leaflet-bar leaflet-control">
                            <button class="all" style="background-color: #7c7c7c;color:#fff; cursor:pointer;" onclick="location.href = '전체.html';">전체</button><br>
                            <button class="kr" onclick="location.href ='한식.html';">한식</button><br>
                            <button class="chi" onclick="location.href = '중식.html';">중식</button><br>
                            <button class="jp" onclick="location.href = '일식.html';">일식</button><br>
                            <button class="we" onclick="location.href = '양식.html';">양식</button><br>
                        </div>
                    </div>
                </div>
                <div class="leaflet-control-container">
                    <div class="leaflet-bottom leaflet-right" style="margin-bottom: 48px ">
                        <div class="leaflet-control-zoom leaflet-bar leaflet-control">        
                            <button class="fa" onclick="location.href = '패스트푸드_치킨.html';">패스트푸드/치킨</button><br>
                            
                        </div>
                    </div>
                </div>
                <div class="leaflet-control-container">
                    <div class="leaflet-bottom leaflet-right" style="margin-bottom: 23px">
                        <div class="leaflet-control-zoom leaflet-bar leaflet-control">        
                            <button class="el" onclick="location.href = '기타.html'">기타</button>
                            
                        </div>
                    </div>
                </div>
                <div class="leaflet-control-container">
                    <div class="leaflet-top leaflet-right">
                        <div class="STAR">
                            <p><Label style="font-size: 15px; font-weight: 900;">별점별 마커</Label>
                            </p>
                            <div class="awesome-marker-icon-red awesome-marker leaflet-zoom-animated leaflet-interactive"
                                tabindex="0" style="margin-top: 40px; margin-left: 17px;">
                                <i class="fa-rotate-0 glyphicon glyphicon-info-sign  icon-white"></i>
                            </div>
                            <div class="awesome-marker-icon-orange awesome-marker leaflet-zoom-animated leaflet-interactive"
                                tabindex="0" style="margin-top: 90px; margin-left: 17px;">
                                <i class="fa-rotate-0 glyphicon glyphicon-info-sign  icon-white">
                                </i>
                            </div>
                            <div class="awesome-marker-icon-gray awesome-marker leaflet-zoom-animated leaflet-interactive"
                                tabindex="0" style="margin-top: 140px; margin-left: 17px;">
                                <i class="fa-rotate-0 glyphicon glyphicon-info-sign  icon-white"></i>
                            </div>
                            <label
                                style="font-size: 14px; font-weight: 900; margin-top: 5px; margin-left: 35px;"><br>&nbsp;&nbsp;&nbsp;&nbsp;3.5~5.0점</label>
                            <br>
                            <label
                                style="font-size: 14px; font-weight: 900; margin-top: 25px; margin-left: 35px;">&nbsp;&nbsp;&nbsp;&nbsp;1.0~3.5점</label>
                            <br>
                            <label
                                style="font-size: 14px; font-weight: 900; margin-top: 20px; margin-left: 35px;">&nbsp;&nbsp;&nbsp;&nbsp;0.0점</label>
                        </div>
                    </div>
                </div>z          